# `MF6ADJ` demonstration using the San Pedro model

In this notebook, we will see now `mf6adj` can be used with an MODFLOW-6 version of the famous San Pedro model of Leake and others (2010)

In [ ]:
import os
import pathlib as pl
import platform
import shutil
import sys
from datetime import datetime

import flopy
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyemu

In [ ]:
sys.path.insert(0, str(pl.Path("../").resolve()))
import mf6adj

First we need to get the platform-specific binaries.  We have some strict control over these and they are stored at the root level in the repo in the `bin` dir.  Let's workout what path we should be using and the binary names we need:

In [ ]:
env_path = pl.Path(os.environ.get("CONDA_PREFIX", None))
assert env_path is not None, "Notebook must be run from the mf6adj Conda environment"

In [ ]:
if "linux" in platform.platform().lower():
    lib_ext = ".so"
    exe_ext = ""
elif "darwin" in platform.platform().lower() or "macos" in platform.platform().lower():
    lib_ext = ".dylib"
    exe_ext = ""
else:
    lib_ext = ".dll"
    exe_ext = ".exe"
lib_name = env_path / f"bin/libmf6{lib_ext}"
mf6_bin = env_path / f"bin/mf6{exe_ext}"

Now let's get the model files we will be using - they are stored in the autotest directory

In [ ]:
org_ws = os.path.join("..", "autotest", "sanpedro", "mf6_transient_ghb")
assert os.path.exists(org_ws)

setup a local copy of the model files.  Also copy in the binaries we need for later....

In [ ]:
ws = "sanpedro"
if os.path.exists(ws):
    shutil.rmtree(ws)
shutil.copytree(org_ws, ws)

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=ws)
m = sim.get_model()

In [ ]:
ib = m.dis.idomain.array.astype(float)
ib[ib > 0] = np.nan
ib_cmap = plt.get_cmap("Greys_r")
ib_cmap.set_bad(alpha=0.0)


def plot_model(k, arr, units=None):
    arr[~np.isnan(ib[k, :, :])] = np.nan
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    cb = ax.imshow(arr, cmap="plasma")
    plt.colorbar(cb, ax=ax, label=units)
    plt.imshow(ib[k, :, :], cmap=ib_cmap)
    return fig, ax

In [ ]:
fig, ax = plot_model(4, m.dis.botm.array[4, :, :])
_ = ax.set_title("botm")

Run the existing model in our local workspace

In [ ]:
pyemu.os_utils.run(mf6_bin.name, cwd=ws)

Now plot some heads...

In [ ]:
hds = flopy.utils.HeadFile(os.path.join(ws, "sp_mf6.hds"))
final_arr = hds.get_data()
fig, ax = plot_model(4, final_arr[4, :, :], units="meters")
ax.set_title("layer 5 final heads")

Ah so nice!  see the sfr "network" running north-south in the 15th column?

The main requirement to use `Mf6Adj` is an input file that describes the performance measures.  Luckily this file has a nice modern format like other MF6 input files.  Here we are going to make one programmatically... `MF6ADJ` supports so-called "flux-based" performance measures, which yield the sensitivity of a simulated flux to the model inputs.  This flux-based performance measure can be described very granularly just like the head-based performance measures.  So lets look at the sensitivity of the simulated sw-gw flux between the groundwater system and sfr across all output times...

In [ ]:
pm_fname = "sfr_perfmeas.dat"
with open(os.path.join(ws, pm_fname), "w") as fpm:
    sfr_data = pd.DataFrame.from_records(m.sfr.packagedata.array)
    fpm.write("begin performance_measure swgw\n")
    for kper in range(sim.tdis.nper.data):
        for kij in sfr_data.cellid.values:
            fpm.write(
                "{0} 1 {1} {2} {3} sfr-1 direct 1.0 -1.0e+30\n".format(
                    kper + 1, kij[0] + 1, kij[1] + 1, kij[2] + 1
                )
            )
    fpm.write("end performance_measure\n\n")

Ok, now we should be ready to go...the adjoint solution process requires running the model forward once and then solving for the adjoint state, which uses the forward solution components (i.e. the conductance matrix, the RHS, heads, saturation,etc). The adjoint state solution has two important characteristics:  its a linear (independent of the forward model's linearity) and it solves backward in time, starting with the last stress period - WAT?!

The adjoint solve is considerably slower than the forward solution, with most of the time being spent in the numpy sparse linear solve...#lyf

In [ ]:
bd = os.getcwd()
os.chdir(ws)

In [ ]:
forward_hdf5_name = "forward.hdf5"
start = datetime.now()

adj = mf6adj.Mf6Adj(pm_fname, lib_name, verbose_level=2)
adj.solve_gwf(hdf5_name=forward_hdf5_name)  # solve the standard forward solution
dfsum = adj.solve_adjoint()  # solve the adjoint state for each performance measure
adj.finalize()  # release components
duration = (datetime.now() - start).total_seconds()
print("took:", duration)

In [ ]:
os.chdir(bd)

Boo ya!  done...let's see what happened...

In [ ]:
[f for f in os.listdir(ws) if f.endswith("hdf5")]

`MF6ADJ` uses the widely available HDF5 format to store information - these files hold very low-level granular information about the adjoint solution.  However the `mf6adj.solve_adjoint()` method also returns a higher-level summary of the adjoint solution.  Let's look at it first:

In [ ]:
type(dfsum)

In [ ]:
list(dfsum.keys())

In [ ]:
dfhw = dfsum["swgw"]
dfhw

those are the node-scale sensitivities to the sfr flux-based performance measure - some plots would be nice you say?!  Well this is most easily done with the HDF5 file itself...

In [ ]:
result_hdf = "adjoint_solution_swgw_forward.hdf5"
hdf = h5py.File(os.path.join(ws, result_hdf), "r")
keys = list(hdf.keys())
keys.sort()
print(keys)

The "composite" group has the sensitivities of the performance measure to the model inputs summed across all adjoint solutions...

In [ ]:
grp = hdf["composite"]
plot_keys = [
    i for i in grp.keys() if len(grp[i].shape) == 3 and ("k33" in i or "wel" in i)
]
plot_keys

A simple routine to plot all these sensitivities....

In [ ]:
for pkey in plot_keys:
    arr = grp[pkey][:]
    for k, karr in enumerate(arr):
        karr[karr == 0.0] = np.nan
        fig, ax = plot_model(k, karr)
        ax.set_title(pkey + ", layer:{0}".format(k + 1), loc="left")

There is one plot in there that is particularly well known with the practice of mapping so-called "capture fraction":

In [ ]:
arr = grp["wel6_q"][3, :, :]
arr[arr == 0.0] = np.nan
fig, ax = plot_model(3, np.abs(arr))
ax.set_title("capture fraction layer 4", loc="left")

What is being shown is the capture fraction: the proportion of groundwater "captured" from the simulated sw-gw flux if a groundwater well was to be added in a given model cell.  Normally, this would be calculated by mechanically adding a wel/specified flux boundary in each model cell, the running the model with this additional boundary cell, and recording how the sw-gw flux changed, and normalizing this change by the rate used in the added boundary cell, repeat for all active cells! - this can take long time to complete.  However, through the magic of the adjoint, the so-called "adjoint state" for this performance measure is simply negative of the capture fraction:  how the simulated gw-sw flux changes as a result of a unit injection of water in each active model cell...